<a href="https://colab.research.google.com/github/mmchenry/canvas/blob/master/polleverywhere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions (when setting this up for a new quarter)
1. In Canvas, make an assignment called polleverywhere that has as many points as the percentage of final grade, which should also be equal the value of polleverywhere scores. 
2. In 'Grades' (in Canvas), select 'Actions: Export' to download a csv file.
3. Rename that file as 'canvas.csv' and move it to the root path (defined below).
4. Open canvas.csv in excel, or other such app, and copy the name of the column that holds your polleverywhere scores. Paste that name in the cell below (in quotes) for the cn_fieldname variable.
5. Place all polleverywhere CSV files in the folder addressed by csv_path in the cell below. Polleverywhere allows exporting in a hierarchical way, so you could have just one file. However, it's probably easier to keep things organized by periodically adding a new CSV file, rather than overwriting the old one with a bigger csv file.
6. In the cell below ("Inititialize packages . . "), update all paths and variables.
7. Run all cells to generate 'canvas_upload.csv'
8. In 'Grades', select 'Actions: Import' to upload the scores and follow the instructions.

# Tips (once set up)
1. Be sure to upload a new canvas.csv file periodically to keep pace with changes in the roster
2. Keep adding CSV files from polleverywhere into the 'polleverywhere CSV files' folder, as they are generated
3. Switching between users will require changing the root path, so comment out the root path and define your own when you switch 
4. Don't forget to download canvas_upload.csv after running the code (give it a minute or two to update first) and then upload it to Canvas to update the scores.
5. Note that the 'Grade' scores from polleverywhere are normalized by the 'participation' score, so it's okay if some of the questions do not end up getting used in class.

# Initialize packages, paths, and parameters

In [8]:
import pandas as pd
import os
import glob
import numpy as np

if 'COLAB_GPU' in os.environ:
    # Mount Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Set up debugger
    %pip install -Uqq ipdb
    import ipdb

    # Debug, when error
    %pdb on

# Change this to the path for the current version of the course
if os.path.isdir('/Users/mmchenry/Documents/Teaching'):
    root = '/Users/mmchenry/Documents/Teaching/E109/2022 Spring E109'

elif os.path.isdir('/content/drive/MyDrive/Teaching'):
    root = '/content/drive/MyDrive/Teaching/E109 Human Physiology/e109 S2022'

elif os.path.isdir('/content/drive/MyDrive/Grad/teaching/e109s22/e109 S2022'):
    root = '/content/drive/MyDrive/Grad/teaching/e109s22/e109 S2022'

else:
    raise ValueError('root path not found. Need to add new root path.')

# Create a folder within root that holds all the csv files from polleverywhere for the quarter
csv_path = root + os.path.sep + "polleverywhere csv files"

# Path to canvas CSV file that includes updated roster
cn_path = root + os.path.sep + "canvas.csv"

# Path to output csv file
out_path = root + os.path.sep + "canvas_upload.csv"

# Proportion of grade earned just from participation
propPart = 0.75

# Value of polleverywhere questions to final grade points
pollValue = 5

# Name of assignment from canvas csv file
cn_fieldname = "polleverywhere (956766)"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Automatic pdb calling has been turned ON


# Calculate polleverywhere scores from csv files, save to canvas csv file

In [17]:
# csv_list = os.listdir(csv_path)

# Open and read canvas data, capture listing of login IDs
cn_file = open(cn_path)
c = pd.read_csv(cn_file)

# From the canvas file, capture student IDs and the polleverywhere data
cnID = c.xs("SIS Login ID",axis=1)
# cnPE = c.xs(cn_fieldname,axis=1)

# Total points earned by each student
tot_earn = np.zeros((len(cnID),1))

# Toal particpation by each student
tot_part = np.zeros((len(cnID),1))

# Listing of csv files
csv_list = glob.glob(csv_path + os.path.sep + '*.csv')

# Loop thru each csv file from polleverywhere
for fPath in csv_list:

    # Open and import data from csv file into d
    file = open(fPath)
    d = pd.read_csv(file)
    file.close()

    # Extract columns
    emails      = d.xs('Email','columns')
    pt_poss     = d.xs('Total points possible','columns')
    pt_earn     = d.xs('Total points earned','columns')

    # Trim final nans
    emails  = emails[:-2]
    pt_poss = pt_poss[:-2]
    pt_earn = pt_earn[:-2]
    
    # Loop trhu IDs in canvas data
    for i in range(2,len(cnID)):
        
        # Index of emails that match current canvas netID
        idx = emails.str.startswith(cnID[i])

        # Check for multiple matches
        if sum(idx)>1:
            raise ValueError('More than one matching UCINetID')

        # Log possible and earned points
        elif sum(idx)==1: 
            tot_earn[i] = tot_earn[i] + float(pt_earn[idx]) 
            tot_part[i] = tot_part[i] + 1

            # Set break point (in Colab)
            # if 'COLAB_GPU' in os.environ:
            #     ipdb.set_trace(context=6)

# # Set break point
# if 'COLAB_GPU' in os.environ:
#     ipdb.set_trace(context=6)

# Participation score
tot_part = tot_part / max(tot_part) * pollValue * propPart

# Correct answer score
tot_earn = tot_earn / max(tot_earn) * pollValue * (1-propPart)

# Transfer values to canvas table
c.loc[:,cn_fieldname] = tot_part + tot_earn

# Transfer only the necessary columns
cOut = c[['Student','ID','SIS User ID','SIS Login ID','Section',cn_fieldname]]

# Write to output file
cOut.to_csv(out_path,index=False)

# Close input csv files
# file.close()
cn_file.close()